# Machine Learning : Premier League

In [1]:
# Importando Bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get('https://www.101greatgoals.com/wp-content/uploads/2017/03/new-premier-league-logo-2016-17-9.jpg')
img = Image.open(BytesIO(response.content))
img

ConnectionError: HTTPSConnectionPool(host='www.101greatgoals.com', port=443): Max retries exceeded with url: /wp-content/uploads/2017/03/new-premier-league-logo-2016-17-9.jpg (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F4BE112EC8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## 1. Introdução

Dentre os campeonatos de futebol mais visualizados do mundo, a Premier League (Campeonato Inglês) tem se destacado por ser um campeonato de grande equilíbrio entre os times. Diferentemente de outros torneios Europeus, em que há majoritariamente a predominância de 1 ou 2 times, como aconteçe na Alemanha, Espanha e Itália, no torneio Inglês acontece uma rotatividade entre os campões nacionacias, como mostra a tabela abaixo:

In [ ]:
df_Ligas = pd.read_excel('Ligas_Campeoes.xlsx')
df_Ligas.index = df_Ligas['Temporada']
df_Ligas.drop(columns = ['Temporada'], inplace = True)
df_Ligas

Devido essa grande disputa, não só ocorre um aumento no público, como  também no número de apostas. Atualmente, é possível encontrar diversos sites de apostas que possuem partidas de futebol disponíveis para dar lances. Número de cartões amarelos, chutes ao gol ou quem será o ganhador da partida são alguns exemplos de apostas disponíveis. A partir dessa infinidade de itens, queremos responder: é possível prever qual será o ganhador de uma partidade de Futebol?


Para tentar responder ess apergunta, utilizou-se a base de dados do kaggle sobre os jogos da Premier League, além das informações sobre folha salarial disponibilizadas pelo site Sportrac.


Kaggle - https://www.kaggle.com/zaeemnalla/premier-league
Sportrac - https://www.spotrac.com/epl/payroll/

## 2. Construindo Banco de Dados

Para construir o modelo de previsão de jogos, teremos as variáveis "Mando da Partida", "Número de Gols produzidos", "Folha Salaria do Time" como dados de entrada. 
É notório que times com maior folha salárial tendem a desempenhar melhor em torneios de pontos corridos, visto que possuem os jogadores mais valiosos.
Também é de se esperar que o fator mando de campo também interferfira no resultado da partida, já que os jogadores terão apoio da torcida.
Por fim, a varíavel de número de gols produzidos é referente a quantos gols aquele time produziu até o momento daquela partida. Se um time faz mais gols em partidas anteriores, em teoria é mais provavel que ele ganhe a partida.

In [ ]:
# Carregando Base de dados Kaggle
df = pd.read_csv('results.csv')
df.head()

In [ ]:
# Alterando Cabeçalhos
dict = {
    'home_team' : 'Mandante',
    'away_team' : 'Visitante',
    'home_goals' : 'Gols_Mandante',
    'away_goals' : 'Gols_Visitante',
    'result': 'Resultado',
    'season': 'Temporada'
}

df = df.rename(columns = dict)
df.head()

Para criação do modelo, vamos trabalhar apenas com os dados de uma temporada, que nesse caso será a 2017-2018.

In [ ]:
# Selecionando apenas temporada 2017-2018
selecao = df['Temporada'] == '2017-2018'
df = df[selecao]
df.head()

In [ ]:
# Corrigindo o Index
index = [i for i in range(df.shape[0])]
index = np.array(index)
df.set_index(index, inplace = True)

In [ ]:
# Adicionando as rodadas
rodada=[j+1 for j in range(38) for i in range(10)]
rodada = np.array(rodada)
df['Rodada'] = rodada
df.head()

### 2.1 Conferência de dados

Como na Premier League possui, 38 rodadas e 20 times, devemos ter 20 times repetindo 19 vezes para a coluna Mandante.

In [ ]:
# Conferência do Número de Times
len(df['Mandante'].unique())

In [ ]:
# Conferência do Número de Rodadas
df['Mandante'].value_counts()

### 2.2 Contruindo coluna Número de Gols 

In [ ]:
# Array Nome Times
array_team_df = df['Mandante'].unique()
array_team_df.sort()
array_team_df

In [ ]:
# Construindo tabela com gols produzidos por cada time por rodada
lista = []
for time in array_team_df:
    selecao = (df['Mandante'] == time) | (df['Visitante'] == time)
    df_selecao = df[selecao]
    for i in range(38):
        if df_selecao.iloc[i,0] == time:
            lista.append(df_selecao.iloc[i,2])
        else:
            lista.append(df_selecao.iloc[i,3])
lista = np.array(lista)
lista = np.split(lista,20)
df_gols = pd.DataFrame(lista)
df_gols = df_gols.T
df_gols.columns = array_team_df

In [ ]:
# Corrindo index
index = np.arange(1,39)
df_gols.set_index(index, inplace = True)
df_gols.columns.name = 'Rodada'
df_gols.head()

In [ ]:
# Somatório de Gols por Rodada
acumulado_gols = []
for time in range (20):
    for rodada in range(38):
        acumulado_gols.append(sum(df_gols.iloc[0:rodada,time]))

acumulado_gols = np.array(acumulado_gols)
acumulado_gols = np.split(acumulado_gols,20)
df_acumulado_gols = pd.DataFrame(acumulado_gols)
df_acumulado_gols = df_acumulado_gols.T
df_acumulado_gols.columns = array_team_df

# Corrigindo Index
index = np.arange(1,39)
df_acumulado_gols.set_index(index, inplace = True)
df_acumulado_gols.columns.name = 'Rodada'

In [ ]:
# Somatório Gols times Mandantes
times = df_acumulado_gols.columns
times = np.array(times)
times = times.tolist()

lista_gols_mandantes = []
for i in range(380):
    nome_time = df['Mandante'][i]
    n_coluna = times.index(nome_time)
    n_linha = df.iloc[i, 6]
    lista_gols_mandantes.append(df_acumulado_gols.iloc[n_linha-1,n_coluna])
df['Acumulado_Gols_Mandante'] = lista_gols_mandantes

# Somatório Gols times Visitantes
lista_gols_visitantes = []
for i in range(380):
    nome_time = df['Visitante'][i]
    n_coluna = times.index(nome_time)
    n_linha = df.iloc[i, 6]
    lista_gols_visitantes.append(df_acumulado_gols.iloc[n_linha-1,n_coluna])
df['Acumulado_Gols_Visitante'] = lista_gols_visitantes
df.tail(10)

Para ajudar o leitor, temos na tabela acima que na Rodada 38, ou seja, até a rodada 37, o time do Manchester City havia produzido 105 gols na temporada, enquanto o time do Everton produziu 37.
Ou seja , nós estamos olhando o histórico de gols produzidos até a rodada analisada para prever o resultado da partida.

In [ ]:
# Colunas Diferenciais
df['Diferença_Gols_Mandante'] = df['Acumulado_Gols_Mandante']-df['Acumulado_Gols_Visitante']
df['Diferença_Gols_Visitante'] = df['Acumulado_Gols_Visitante']-df['Acumulado_Gols_Mandante']
df.head()

### 2.3. Base de Dados Salário

In [ ]:
# Carregando Salários
df_Salary = pd.read_excel('Salary.xlsx')
df_Salary.sort_values(by = 'TEAM', inplace = True)
df_Salary.head()

In [ ]:
# Array de Times 
array_team_df_Salary = df_Salary['TEAM'].unique()
array_team_df_Salary.sort()
array_team_df_Salary

In [ ]:
# Comparação de nomes
df_times_names = pd.DataFrame({'Kaggle':array_team_df, 'Spotrac': array_team_df_Salary})
df_times_names

In [ ]:
# Criando Relação
relacao = np.array([array_team_df, array_team_df_Salary]) 

# Criando Dicionário
dict = {}
for j in range(relacao.shape[1]):
    dict[relacao[0,j]] = relacao[1,j]
    
print(dict)

In [ ]:
# Substituindo nome em df
df['Mandantes'] = df.Mandante.map(dict)
df['Visitantes'] = df.Visitante.map(dict)
df.drop(columns = ['Mandante','Visitante', 'Gols_Mandante', 'Gols_Visitante'], inplace = True)
df.head()

In [ ]:
# Dicionario Folha Salário
dict_salary = {}
for i in range(len(array_team_df_Salary)):
    dict_salary[array_team_df_Salary[i]] = df_Salary.iloc[i,3]
print (dict_salary)

In [ ]:
# Construindo Coluna de Fola Salarial
df['Folha_Mandante'] = df.Mandantes.map(dict_salary)
df['Folha_Visitante'] = df.Visitantes.map(dict_salary)

In [ ]:
# Diferença Salarial
df['D_Folha_Mandante'] = df['Folha_Mandante'] - df['Folha_Visitante']
df['D_Folha_Visitante'] = df['Folha_Visitante'] - df['Folha_Mandante']
df.head()

### 2.4 Atribuindo placar

In [ ]:
# Resultado
dict_resultado_mandante = {
    'H':2,
    'D':1,
    'A':0
}
dict_resultado_visitante = {
    'H':0,
    'D':1,
    'A':2
}
df['Score_Mandante'] = df.Resultado.map(dict_resultado_mandante)
df['Score_Visitante'] = df.Resultado.map(dict_resultado_visitante)
df.head()

### 2.5 Empilhando dados

Para emplihar os dados, vamos selecionar times que jogaram em casa em 50 % das partidas e o 50% times que jogaram fora de casa nas partidas. Para facilitar,vamos determinar que caso a coluna possua um index par, analizaremos os times que estão jogando em casa. Para o index impar, analisarem os time visitante.

In [ ]:
# Colunas Necessárias Casa, D_Folha_Casa, Result, selecionando metade dos dados (Colunas Pares)
df_ML = df[['Rodada', 'Mandantes', 'Acumulado_Gols_Mandante', 'Diferença_Gols_Mandante', 'Folha_Mandante', 'D_Folha_Mandante', 'Score_Mandante']][df.index%2 == 0]
df_ML['Mando_de_Campo'] = 1
df_ML.columns = ['Rodada','Time', 'Acumulado_Gols', 'Diferença_Gols', 'Folha', 'Diferença_Folha', 'Score', 'Mando_Campo']
df_ML.head()

In [ ]:
# Colunas Necessárias Casa, D_Folha_Casa, Result, selecionando metade dos dados (Colunas Pares)
df_ML_1 = df[['Rodada', 'Visitantes', 'Acumulado_Gols_Visitante', 'Diferença_Gols_Visitante', 'Folha_Visitante', 'D_Folha_Visitante', 'Score_Visitante']][df.index%2 == 1]
df_ML_1['Mando_de_Campo'] = 0
df_ML_1.columns = ['Rodada','Time', 'Acumulado_Gols', 'Diferença_Gols', 'Folha', 'Diferença_Folha', 'Score', 'Mando_Campo']
df_ML_1.head()

In [ ]:
# Empilhando dados
Consolidado = [df_ML,df_ML_1]
Resultado = pd.concat(Consolidado)
Resultado.head()

## 3. Criando Modelo de MachineLearning

### 3.1. Analise Comportamento de Dados

Para entender como é o comportamento das 3 variáveis analisadas vamos contruir gráficos de dispersão. Lembrando que os números 0,1,2 representam Derrota, Empate e Vitória respectivamente.

In [ ]:
plt.rc('figure', figsize = (15,14))
fig, ax =plt.subplots(3,1)

sns.scatterplot(x='Diferença_Folha', y='Diferença_Gols',  hue='Score', data=Resultado, ax=ax[0]) 
sns.scatterplot(x='Diferença_Folha', y='Mando_Campo',  hue='Score', data=Resultado, ax=ax[1]) 
sns.scatterplot(x='Diferença_Gols', y='Mando_Campo',  hue='Score', data=Resultado, ax=ax[2])

O gráfico 1 reafirma a importância dos número de gols e folha salarial, já que há uma concentração de bolas pretas (Vitórias) no canto superior esquerdo do gráfico. O mando de campo também demonstrar ser um atributo relevante, visto existe um maior concentração de bolas pretas(Vitórias) em situações em que o time foi o mandante.

### 3.2. Modelo Linear SVC

### 3.2.1. Definindo Variáveis

Em um primeiro momento vamos utilizar apenas os atributos 'Diferença_Gols' e 'Diferença_Folha', para que possamos desenhar a barreira de decisão.

In [ ]:
X = Resultado[['Diferença_Gols', 'Diferença_Folha']]
y = Resultado['Score']

### 3.2.2. Contruindo Modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

# Definido semente Padrão
SEED = 39
np.random.seed(SEED)

# Dividindo amostra em treino e teste
treino_x, teste_x, treino_y, teste_y = train_test_split(X, y,
                                                         test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

# Criando modelo 
modelo = LinearSVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

# Calculando Acuracia do Modelo
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

### 3.2.3. Contruindo Barreira de Decisão 

In [ ]:
x_min = teste_x.Diferença_Gols.min()
x_max = teste_x.Diferença_Gols.max()
y_min = teste_x.Diferença_Folha.min()
y_max = teste_x.Diferença_Folha.max()

pixels = 100
eixo_x = np.arange(x_min, x_max, (x_max - x_min) / pixels)
eixo_y = np.arange(y_min, y_max, (y_max - y_min) / pixels)

xx, yy = np.meshgrid(eixo_x, eixo_y)
pontos = np.c_[xx.ravel(), yy.ravel()]

Z = modelo.predict(pontos)
Z = Z.reshape(xx.shape)

import matplotlib.pyplot as plt

plt.contourf(xx, yy, Z, alpha=0.3)
ax = plt.scatter(teste_x.Diferença_Gols, teste_x.Diferença_Folha, c=teste_y, s=20)

Apenas pela imagem acima, é possível verificar que a decisão do modelo foi quase igual a dividir a amortra no meio, e prever vitória para quem estiver acima e derrota para os que estiverem abaixo. A acurácia foi igual a 23,16%, ou seja, aparentemente ínfima.
Mas quão ruim foi a previsão desse modelo? Necessitamos ter uma base para saber classificar o nosso resultado.

### 3.2.4. Dummy Classifier

Esse estimador irá prever qual é acurácia do modelo caso optamos por chutar o valor que mais se repete (nesse caso foi a Vitória). Isto é, prevendo que todos os times analisado fossem sair vitoriosos das partidas, qual seria o percentual de acerto do nosso modelo?

In [ ]:
# Comparando a acurácia
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(treino_x, treino_y)
acuracia = dummy.score(teste_x, teste_y) * 100

print("A acurácia do dummy foi %.2f%%" % acuracia)

### 3.3. Modelo SVC

### 3.3.1. Definindo Variáveis

In [ ]:
X = Resultado[['Diferença_Gols', 'Diferença_Folha']]
y = Resultado['Score']

### 3.3.2. Construindo Modelo

In [ ]:
# Utilizando modelo Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Definido semente Padrão
SEED = 39
np.random.seed(SEED)

# Dividindo amostra em treino e teste
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(raw_treino_x), len(raw_teste_x)))

# Normalizador
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

# Criando modelo
modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

# Calcuando acurácia do Modelo
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


### 3.3.3. Construindo Barreira de Decisão

In [ ]:
data_x = teste_x[:,0]
data_y = teste_x[:,1]

x_min = data_x.min()
x_max = data_x.max()
y_min = data_y.min()
y_max = data_y.max()

pixels = 100
eixo_x = np.arange(x_min, x_max, (x_max - x_min) / pixels)
eixo_y = np.arange(y_min, y_max, (y_max - y_min) / pixels)

xx, yy = np.meshgrid(eixo_x, eixo_y)
pontos = np.c_[xx.ravel(), yy.ravel()]

Z = modelo.predict(pontos)
Z = Z.reshape(xx.shape)

import matplotlib.pyplot as plt

plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(data_x, data_y, c=teste_y, s=20)

No gráfico acima, conseguimos perceber a diferença do modelo SVC, em que o modelo não fica restrito ao traado de linhas. É possível verificar 3 áreas claras de tomada de decisão, sendo roxa (derrota), verde (empate), amarela (Vitória). Como foi possível observar seção do comportamento de dados, quanto mais acima e a direita do gráfico, mais proválvel é a vitória. O oposto também segue a lógica. E na região central do gráfico o modelo definiu a área de empate. Entretanto, ainda que a acurácia teha subido para 50,53% ainda é elevado o número de pontos de coloração diferente da área que está incluído.

Logo, iremos utilizar a variável 'Mando_Campo' tentar aumentar a acurácia.

### 3.3.4 Adiconando 3° Atributo

In [ ]:
# Definindo X e y
X = Resultado[['Diferença_Gols', 'Diferença_Folha', 'Mando_Campo']]
y = Resultado['Score']

In [ ]:
# Utilizando modelo Machine Learning
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Definido semente Padrão
SEED = 39
np.random.seed(SEED)

# Dividindo amostra em treino e teste
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.25,
                                                         stratify = y)

# Normalizador
scaler = StandardScaler()
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

# Criando modelo
modelo = SVC()
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

# Calcuando acurácia do Modelo
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia teve um aumento considerável ao introduzirmos o atributo mando de campo. No entanto, com essa adição não é possível desenha o gráfico da barreira de decissão. Ou seja, não conseguimos vizualizar de que forma o modelo faz a tomada de decisão.

### 3.4. Modelo Árvore de Decisão

### 3.4.1. Definindo Variáveis

In [ ]:
# Definindo X e y
X = Resultado[['Diferença_Gols', 'Diferença_Folha', 'Mando_Campo']]
y = Resultado['Score']

### 3.4.2. Construindo Modelo

In [ ]:
# Utilizando modelo Machine Learning
from sklearn.tree import DecisionTreeClassifier

# Definido semente Padrão
SEED = 39
np.random.seed(SEED)

# Dividindo amostra em treino e teste
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(X, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

# Criando modelo
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(raw_treino_x, treino_y)
previsoes = modelo.predict(raw_teste_x)

# Calcuando acurácia do Modelo
acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


### 3.4.3. Árvore de Decisão

In [ ]:
from sklearn.tree import export_graphviz
import graphviz

features = X.columns
dot_data = export_graphviz(modelo, out_file=None,
                           filled = True, rounded = True,
                           feature_names = features,
                          class_names = ["Derrota", "Empate","Vitória"])
grafico = graphviz.Source(dot_data)
grafico

## 5. Comparação de Modelos

In [ ]:
df_Modelos = pd.read_excel('Modelos.xlsx')
df_Modelos.index = df_Modelos['Modelo']
df_Modelos.drop(columns = ['Modelo'], inplace = True)
df_Modelos

## 6. Conclusão

Embora o modelo SVC tenha apresentado melhor acurácia (57,89%) , ao utilizar essa ferramenta com >2 variáveis, não é possível traçar o gráfico de barreira de decisão. Ou seja, caímos em um blackbox, em que não é possível ratrear como foi escolhida a tomada de decisão. 

Para aumentar a acurácia do modelo acredito que ouras variáveis de entrada podem ser acrescentadas, como número de gols sofridos por time (ponderando as melhores defesas) e valor do time que entrou em campo (ponderar a situação em que reservas são utilizados).